In [ ]:
#Steven Marsh
#The purpose of this code is to help with sovling the issue of trying to find a large amount of Powerdesigner files
#

import os.path, time
import csv
import xml.etree.ElementTree as ET
import pandas as pd

#I use the Python set to store multiple items in a single variable. This will be changed per user. 
excludes = set(['archive', ;'Archive', 'Example_directory_names'])
#pd_dir was created to save time by staring it at the directory folder and go from there.
#This also works with cloud stored directories for a company.
pd_dir = r'This will depend on the directory you are working with on.'
#bad_files is to store a list of pd files that cannot be parsed as xml.
bad_files = []
#housekeeping_report data will contain the final product.
housekeeping_report_list = list()
file_count = 0

#Powerdesigner files to find.
for root, dirs, files in os.walk(pd_dir):
    #I put in logic for it to avoid certain directories.
    dirs[:] = [d for d in dirs if d not in excludes]
    for file in files:
        #endswith is a function in Python
        if file.endswith('.ldm') or file.endswith('.cdm') or file.endswith('.pdm'):
            #Capture the file path.
            full_file_name = os.path.join(root, file)
            
            #Getting the Powerdesigner modified date. 
            pd_modified_time = os.path.getmtime(os.path.join(root,file))
            pd_modified_date = time.strftime('%Y-%m-%d',time.localtime(pd_modified_time))
            
            #Attempt to parse pd file as xml, catch it if it fails.
            #This is for when it tries to read a Powerdesigner file that was saved as binary.
            try:
                tree = ET.parse(full_file_name)
            except ET.ParseError:
                bad_files.append(os.path.join(root, file))
                continue
                
            #This returns the root element of a tree.
            #The code below is things that are specific to Powerdesigner. Depending on what the user is trying
            #to do this code will have to be slightly changed. What I was doing here was while I am finding the
            #desired files we get certain objects that are in the .text and store them as variables to put in the 
            #housekeeping_report_list
            gotten_root = tree.getroot()
            for root_object in gotten_root.findall('.//{object}RootObject'):
                model_object = root_object.find('.//{object}Model')
                model_id = model_object.find('.//{attribute}ObjectID').text
                employee_badge = model_object.find('.//{attribute}Modifier').text
            #Looping through the reports in the model
            for intramodelreport_object in model_object.findall('.//{object}IntraModelReport'):
                this_report_name = intramodelreport_object.find('.//{attribute}Name').text
                
                
            housekeeping_report_list.append((full_file_name,model_id,pd_modified_date, employee_badge,model_object))